# Water Pumps - Modeling Pipeline

In this notebook, I will take what I learned in the Modeling notebook to create a pipeline that fits all three models. The notebook can be configured to either over sample or under sample the data. For each model, hyperparameter tuning will be performed, using the strategy developed in the Modeling notebook. The best parameters will then be selected for the final model for each model type. All results are saved to a csv file. At the end of the notebook, the results between each model can be compared.

## Import Libraries

In [1]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

## Configurations
Set up configurations for current run of this notebook.

In [392]:
binary = False
debug = False
sampling_type = 'over'
# sampling_type = 'under'
results_filename = 'pipeline_results.csv'
if binary:
    split_filename = results_filename.split('_')
    results_filename = '_'.join([split_filename[0], 'binary', split_filename[1]])
results_filepath = f'../results/{results_filename}'

## Load Train and Test Sets
In order to ensure an equal comparison between the baseline model and the additional models developed here, the same training data as the baseline model will be used.

In [393]:
def load_train_test():
    file_list = ['X_train', 'X_test', 'y_train', 'y_test']
    data_sets = []
    for filename in file_list:
        data_sets.append(pickle.load(open(f'../data/clean/{filename}', 'rb')))
    return tuple(data_sets)

In [394]:
X_train, X_test, y_train, y_test = load_train_test()

In [395]:
# Use this block to shortening training data for debugging.
if debug:
    row_cut = 100
    X_train = X_train[:100]
    y_train = y_train[:100]

### Baseline Model
Load predictions from baseline model.

In [396]:
y_pred_base = pickle.load(open(f'../data/clean/y_pred_base', 'rb'))

## Prepare Training Data
Data preparations includes:
* Scaling the data to values between 0 and 1.
* Resampling the data. Either:
    * Over sampling.
    * Under sampling.

In [397]:
def prepare_data(X_train, X_test, y_train, y_test, sampling_type):
    
    if binary:
        y_train = pd.Series(y_train).replace({'functional needs repair': 'faulty', 'non functional': 'faulty'}).values
        y_test = pd.Series(y_test).replace({'functional needs repair': 'faulty', 'non functional': 'faulty'}).values
    
    scaler = MinMaxScaler().fit(X_train)
    X_train_rescaled = scaler.transform(X_train)
    X_test_pipe = scaler.transform(X_test)
    
    if sampling_type == 'over':
        X_train_pipe, y_train_pipe = SMOTE().fit_resample(X_train_rescaled, y_train)
    elif sampling_type == 'under':
        X_train_pipe, y_train_pipe = RandomUnderSampler(random_state=42).fit_resample(X_train_rescaled, y_train)
    else:
        raise Exception("sampling_type must be 'over' or 'under'. Please try again.")
    
    y_test_pipe = y_test
    
    return X_train_pipe, X_test_pipe, y_train_pipe, y_test_pipe

In [398]:
X_train_pipe, X_test_pipe, y_train_pipe, y_test_pipe = prepare_data(X_train, X_test, y_train, y_test, sampling_type)

## Modeling
Here I will try three different models:
* Logistic Regression
* Random Forest
* XGBoost

### Load Saved Results
If a results file exists, load it, otherwise set it to `None`.

In [399]:
if os.path.isfile(results_filepath):
    df_results = pd.read_csv(results_filepath, index_col=0, header=[0, 1])
    df_results.drop('class count', level=1, axis=1, inplace=True)
else:
    df_results = None

In [400]:
df_results

base_line           logreg_under            rf_under  \
                        precision    recall    precision    recall precision   
functional               0.767559  0.911198     0.838948  0.637876  0.851116   
functional needs repair  0.632258  0.151938     0.218284  0.725581  0.259851   
non functional           0.787948  0.659432     0.717764  0.668050  0.715637   

                                  xgb_under            
                           recall precision    recall  
functional               0.648719  0.852985  0.646476  
functional needs repair  0.756589  0.235776  0.758140  
non functional           0.724545  0.737579  0.706033

In [401]:
def store_results(y_test, y_pred, model_type, df=None):
    
    if df is not None:
        if model_type in df.columns:
            df.drop(model_type, level=0, axis=1, inplace=True)
    else:
        pass
    
    results = classification_report(y_test, y_pred, output_dict=True)
    df_results = pd.DataFrame(results).T
    df_results.drop(columns=['f1-score', 'support'], inplace=True)
    df_results.drop(['accuracy', 'macro avg', 'weighted avg'], inplace=True)
    
    multi_columns = [(model_type, x) for x in df_results.columns]
    df_results.columns = pd.MultiIndex.from_tuples(multi_columns)
    
    if df is None:
        return df_results
    else:
        df_results = pd.concat([df, df_results], axis=1)
        df_results.sort_index(axis=1, level=0, inplace=True)
        return df_results

### Baseline Model
Set up baseline model results and store in data frame.

In [402]:
model_type = 'base_line'

In [403]:
if binary == False:
    df_results = store_results(y_test, y_pred_base, model_type, df=df_results)
    df_results

### Logistic Regression

In [404]:
model_type = f'logreg_{sampling_type}'

In [405]:
logreg_rs = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=10000)
rs_logreg_params = {'C': np.arange(0.2, 2.4, 0.4), 'penalty': ['l1', 'l2']}
rs_logreg = RandomizedSearchCV(logreg_rs, rs_logreg_params, random_state=42, n_jobs=-1)
rs_logreg.fit(X_train_pipe, y_train_pipe)

RandomizedSearchCV(estimator=LogisticRegression(max_iter=10000,
                                                multi_class='multinomial',
                                                solver='saga'),
                   n_jobs=-1,
                   param_distributions={'C': array([0.2, 0.6, 1. , 1.4, 1.8, 2.2]),
                                        'penalty': ['l1', 'l2']},
                   random_state=42)

In [406]:
def print_best_params(best_params):
    for key, value in best_params.items():
        if type(best_params[key]) == str:
            print(f' * {key}: {best_params[key]}')
        else:
            print(f' * {key}: {best_params[key]:0.3f}')

In [407]:
print('The best hyperparameters of logistic regression are:')
print_best_params(rs_logreg.best_params_)

The best hyperparameters of logistic regression are:
 * penalty: l1
 * C: 2.200


In [408]:
def save_hyper_params(model_type, sampling, binary, best_params):
    
    next_row = {
        'sampling': sampling,
        'num_classes': 3 - int(True == binary),
    }
    
    for key, value in best_params.items():
        next_row[key] = best_params[key]
    
    hyper_params_files = f'../results/{model_type}_hyperparams.csv'
    if os.path.isfile(hyper_params_files):
        df = pd.read_csv(hyper_params_files)
        next_index = len(df)
        df.loc[next_index] = next_row
        df.drop_duplicates(subset=['sampling', 'num_classes'], ignore_index=True, inplace=True, keep='last')
    else:
        df = pd.DataFrame(next_row, index=[0])
    df.to_csv(hyper_params_files, index=False)
    
    return df

In [409]:
df_logreg_params = save_hyper_params('logreg', sampling_type, binary, rs_logreg.best_params_)

In [411]:
logreg_best = LogisticRegression(
    solver='saga', 
    multi_class='multinomial', 
    C=rs_logreg.best_params_['C'], 
    penalty=rs_logreg.best_params_['penalty'], 
    max_iter=10000
)
logreg_best.fit(X_train_pipe, y_train_pipe)

LogisticRegression(C=2.2000000000000006, max_iter=10000,
                   multi_class='multinomial', penalty='l1', solver='saga')

In [412]:
y_pred_logreg_best = logreg_best.predict(X_test_pipe)

In [413]:
df_results = store_results(y_test_pipe, y_pred_logreg_best, model_type, df=df_results)

### Random Forest

In [415]:
model_type = f'rf_{sampling_type}'

In [416]:
rf_rs = RandomForestClassifier(random_state = 42, n_jobs=-1)

In [417]:
max_depth_list = list(np.arange(10, 110, 10))
max_depth_list.append(None)

In [418]:
rs_rf_params = {
    'bootstrap': [True, False],
    'max_depth': max_depth_list,
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': list(np.arange(1, 11, 1)),
    'min_samples_split': list(np.arange(1, 11, 1)),
    'n_estimators': list(np.arange(200, 2200, 200))
}

In [419]:
rs_rf = RandomizedSearchCV(rf_rs, rs_rf_params, random_state=42, n_jobs=-1)
rs_rf.fit(X_train_pipe, y_train_pipe)

RandomizedSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42)

In [420]:
print('The best hyperparameters for random forest are:')
print_best_params(rs_rf.best_params_)

The best hyperparameters for random forest are:
 * n_estimators: 800.000
 * min_samples_split: 3.000
 * min_samples_leaf: 1.000
 * max_features: auto
 * max_depth: 90.000
 * bootstrap: 1.000


Double the number of trees for the final random forest model.

In [421]:
rs_rf.best_params_['n_estimators'] *= 2

In [422]:
df_rf_params = save_hyper_params('rf', sampling_type, binary, rs_rf.best_params_)

In [424]:
rf_best = RandomForestClassifier(
    n_estimators = rs_rf.best_params_['n_estimators'],
    min_samples_split = rs_rf.best_params_['min_samples_split'],
    min_samples_leaf = rs_rf.best_params_['min_samples_leaf'],
    max_features = rs_rf.best_params_['max_features'],
    max_depth = rs_rf.best_params_['max_depth'],
    bootstrap = rs_rf.best_params_['bootstrap'],
    random_state = 42, 
    n_jobs=-1
)

In [425]:
rf_best.fit(X_train_pipe, y_train_pipe)

RandomForestClassifier(max_depth=90, min_samples_split=3, n_estimators=1600,
                       n_jobs=-1, random_state=42)

In [426]:
y_pred_rf = rf_best.predict(X_test_pipe)

In [427]:
df_results = store_results(y_test_pipe, y_pred_rf, model_type, df=df_results)

### XGBoost

In [429]:
model_type = f'xgb_{sampling_type}'

In [430]:
if binary:
    class_mapping = {
        'functional': 0,
        'faulty': 1
    }
else:
    class_mapping = {
        'functional': 0,
        'functional needs repair': 1,
        'non functional': 2
    }

In [431]:
y_train_encoded = pd.Series(y_train_pipe).replace(class_mapping).values
y_test_encoded = pd.Series(y_test_pipe).replace(class_mapping).values

In [432]:
xgb_rs = XGBClassifier(n_estimators=1000)

In [433]:
if debug:
    rs_params_xgb_over = {
        'max_depth': [1],
        'min_child_weight': [1],
        'gamma': [0]
    }
else:
    rs_params_xgb_over = {
        'max_depth': list(np.arange(1, 7, 2)),
        'min_child_weight': list(np.arange(1, 7, 2)),
        'gamma': [0, 1, 5]
    }

In [434]:
rs_xgb = RandomizedSearchCV(xgb_rs, rs_params_xgb_over, random_state=42, n_jobs=-1, n_iter=100)
rs_xgb.fit(X_train_pipe, y_train_encoded)

/Users/dave/anaconda3/envs/springboard/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 27 is smaller than n_iter=100. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/dave/anaconda3/envs/springboard/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:01:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=1000, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                           reg_lam

In [435]:
print('The best parameters for XGBoost are:')
print_best_params(rs_xgb.best_params_)

The best parameters for XGBoost are:
 * min_child_weight: 1.000
 * max_depth: 5.000
 * gamma: 0.000


A larger number of trees should improve performance. Set the total number of trees to 1000.

In [436]:
rs_xgb.best_params_['n_estimators'] = 1000

In [437]:
df_xgb_params = save_hyper_params('xgb', sampling_type, binary, rs_xgb.best_params_)

In [439]:
xgb_best = XGBClassifier(
    n_estimators=rs_xgb.best_params_['n_estimators'],
    max_depth=rs_xgb.best_params_['max_depth'],
    min_child_weight=rs_xgb.best_params_['min_child_weight'],
    gamma=rs_xgb.best_params_['gamma']
)

In [440]:
xgb_best.fit(X_train_pipe, y_train_encoded)

/Users/dave/anaconda3/envs/springboard/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:03:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [441]:
y_pred_encoded = xgb_best.predict(X_test_pipe)

In [442]:
reverse_mapping = {}
if binary:
    reverse_mapping = {
        0:'functional',
        1:'faulty'
    }
else:
    reverse_mapping = {
        0:'functional',
        1:'functional needs repair',
        2:'non functional'
    }

In [443]:
y_pred_xgb = pd.Series(y_pred_encoded).replace(reverse_mapping).values

In [444]:
df_results = store_results(y_test_pipe, y_pred_xgb, model_type, df=df_results)

### Add Class Counts

In [445]:
def add_class_counts(df, y_test):
    df_class_counts = pd.DataFrame(pd.Series(y_test).value_counts())
    df_class_counts.columns = pd.MultiIndex.from_tuples([('', 'class count')])
    df_results = pd.concat([df, df_class_counts], axis=1)
    return df_results

In [446]:
df_results = add_class_counts(df_results, y_test_pipe)

### Save Results
Save the results from the pipeline to the results directory.

In [448]:
df_results.to_csv(results_filepath)

### Display Results

In [449]:
def get_results(get_binary=False):
    results_filename = 'pipeline_results.csv'
    if get_binary:
        split_filename = results_filename.split('_')
        results_filename = '_'.join([split_filename[0], 'binary', split_filename[1]])
    results_filepath = f'../results/{results_filename}'
    if os.path.isfile(results_filepath):
        df = pd.read_csv(results_filepath, index_col=0, header=[0, 1])
        unnnamed = [x[0] for x in df.columns if 'Unnamed' in x[0]][0]
        new_index = [(x[0].replace(unnnamed, ' '), x[1]) if unnnamed in x else x for x in df.columns.values.tolist()]
        df.columns = pd.MultiIndex.from_tuples(new_index)
        return df
    else:
        return None

In [450]:
df_final_results = get_results(get_binary=False)

In [451]:
df_final_binary_results = get_results(get_binary=True)

### Final Results:

I carried out modeling to predict water pump functional status. The response variable consisted of three classes:
* functional
* functional needs repair
* non functional

Three different algorithms were tried:
* Logistic Regression
* Random Forest
* XGBoost

All three models were compared to a baseline model. The baseline model used logistic regression with the no regularization.

Each model was fit to the same test set. I used a train-test split of 30% test / 70% train. To improve model training speed, the train and test data were scaled to values between 0 and 1. As noted during preprosing, I observed a large class imbalance between each class. Therefore, I tried two resampling methods, over sampling and under sampling, to see if that improved the model performance.

The goal of the modeling was to achieve a high recall score for the minority classes, which were the classes with faulty pumps.

#### All Three Classes:
I began with training each model to predict each of the three classes. In the below table I sumarise my results.

In [452]:
df_final_results

base_line           logreg_over            \
                        precision    recall   precision    recall   
functional               0.767559  0.911198    0.845651  0.652458   
functional needs repair  0.632258  0.151938    0.225578  0.741085   
non functional           0.787948  0.659432    0.733426  0.674433   

                        logreg_under             rf_over            rf_under  \
                           precision    recall precision    recall precision   
functional                  0.838948  0.637876  0.834833  0.867452  0.851116   
functional needs repair     0.218284  0.725581  0.466899  0.415504  0.259851   
non functional              0.717764  0.668050  0.813356  0.777530  0.715637   

                                   xgb_over           xgb_under            \
                           recall precision    recall precision    recall   
functional               0.648719  0.825182  0.867452  0.852985  0.646476   
functional needs repair  0.756589  0.462963  0.387597  0.235776  0.758140   
non functional           0.724545  0.802632  0.759336  0.737579  0.706033   

                                     
                        class count  
functional                     5349  
functional needs repair         645  
non functional                 3133

**Observations:**
* All three models perform better in recall as compared to the baseline model.
* The best model was created using XGBoost with undersampling.
    * The highest recall score was 0.76.

While the models did perform better than the baseline model, a recall score of 0.76 is still pretty low. So, I tried modeling the data again using two classes.

##### Hyperparameters
Here are the hyperparameters that were used for each model:

* Logistic Regression

In [460]:
df_logreg_params[df_logreg_params['num_classes'] == 3].reset_index(drop=True)

,sampling,num_classes,penalty,C
0,under,3,l2,2.2
1,over,3,l1,2.2


* Random Forest

In [461]:
df_rf_params[df_rf_params['num_classes'] == 3].reset_index(drop=True)

,sampling,num_classes,n_estimators,min_samples_split,min_samples_leaf,max_features,max_depth,bootstrap
0,under,3,1600,3,1,auto,90,True
1,over,3,1600,3,1,auto,90,True


* XGBoost

In [462]:
df_xgb_params[df_xgb_params['num_classes'] == 3].reset_index(drop=True)

,sampling,num_classes,min_child_weight,max_depth,gamma,n_estimators
0,under,3,1,5,1,1000
1,over,3,1,5,0,1000


#### Two Classes:
I combined the minority classes _functional needs repair_ and _non functional_ into a single class called _faulty_. The goal now was simplified to identify good and problematic.

Here are the results of the binary classification problem:

In [453]:
df_final_binary_results

logreg_over           logreg_under             rf_over            \
             precision    recall    precision    recall precision    recall   
faulty        0.731618  0.737427     0.727178  0.742192  0.805525  0.764161   
functional    0.813499  0.808936     0.815215  0.803328  0.839257  0.869695   

            rf_under            xgb_over           xgb_under            \
           precision    recall precision    recall precision    recall   
faulty      0.764766  0.798571  0.794540  0.747221  0.752093  0.784542   
functional  0.853146  0.826510  0.828669  0.863526  0.843039  0.817349   

                        
           class count  
faulty            3778  
functional        5349

**Observations:**
* The best model was random forest, using undersampling.
    * The highest recall value was 0.80.

#### Hyperparameters
This is a summary of the hyperparameters use for each binary classification model.

* Logistic Regression:

In [463]:
df_logreg_params[df_logreg_params['num_classes'] == 2].reset_index(drop=True)

,sampling,num_classes,penalty,C
0,under,2,l2,1.8
1,over,2,l1,1.0


* Random Forest

In [464]:
df_rf_params[df_rf_params['num_classes'] == 2].reset_index(drop=True)

,sampling,num_classes,n_estimators,min_samples_split,min_samples_leaf,max_features,max_depth,bootstrap
0,under,2,3600,6,2,auto,90,False
1,over,2,1600,3,1,auto,90,True


* XGBoost

In [465]:
df_xgb_params[df_xgb_params['num_classes'] == 2].reset_index(drop=True)

,sampling,num_classes,min_child_weight,max_depth,gamma,n_estimators
0,under,2,1,3,0,1000
1,over,2,1,5,0,1000


## Conclusions
I was able to achieve a recall score of 0.80 on the minority class (_faulty_). The best algorithm was random forest, using an under sampled training set to balance the classes. Also, the two minority classes, _non functional_ and _functional needs repair_ were combined into a single class, _faulty_.

## Recommendation to Client
I have created a model that can accurately identify faulty pumps. By optimizing on recall, I chose to reduce the error of predicting a pump is _functional_, when in fact it is _faulty_. I believe this model can be an accurate tool for the Tanzanian government in there effort to monitor pump status in the country. It should greatly reduce the need to determine the pump status in person and can instead rely on pump data collected on line. 

## Future Directions
While I have achieved an acceptable model for the Tanzanian government, there are a few areas I would like to improve on during updated versions of the project. This includes:
* Try additional sampling methods to counter class imbalance, aside from the two I tried here.
* Perform a more detailed exploration of hyperparameter values. A more granualar search could lead to more refined models with better results.
* Try a different train/test split. A smaller split could give the models more data to train on, which could possibly improve the predictions.